In [1]:
"""
FUNDAMENTAL
TECHNICAL 
    (LINEAR_COEF)
TRADER
"""

'\nFUNDAMENTAL\nTECHNICAL \n    (LINEAR_COEF)\nTRADER\n'

In [2]:
import datetime as dt
import pandas as pd

from de.processor.dart import DART_FUNDAMENTAL_PROCESSOR, DART_INFO_PROCESSOR
from de.processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

from de.loader.pykrx_loader import get_pykrx_loader

from de.processor.factor.fundamental import FUNDAMENTAL_FACTOR_PROCESSOR
from de.processor.factor.technical import LINEAR_COEFFICENT_FACTOR_PROCESSOR
from de.processor.factor.trader import TRADER_FACTOR_PROCESSOR

from de.processor.factor.filter import FACTOR_FILTER

In [3]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
        "FILTER": {
            "quantile_conditions": [
                ("CLR", 0.0, 1.0),
                ("TLR", 0.1, 0.7),
                ("NPP", 0.3, 0.8),
                ("OPP", 0.3, 0.7),
                ("TAPP", 0.0, 0.5),
                ("TEPP", 0.0, 0.5),
                ("CAPP", 0.0, 0.4),
                ("CEPP", 0.1, 0.5),
            ]
        },
    },
    "LINEAR_COEF": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "moving_average_window": 10,
            "coef_recent_n": 20,
        },
        "FILTER": {
            "quantile_conditions": [
                ("VolumeRotation", 0.4, 0.7),
                ("Close", 0.0, 0.3),
            ],
        },
    },
    "TRADER": {
        "PROCESSOR": {
            "start": (dt.date.today() - dt.timedelta(days=7)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FILTER": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
}

In [4]:
# Load

In [5]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [6]:
# dart_info
dart_info_df = pd.read_csv("./de/data/dart_info_df.csv", index_col=0)
dart_info_processor = DART_INFO_PROCESSOR(dart_info_df)

In [7]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [8]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [9]:
# Model

In [10]:
# Fundamental
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(ohlcv_df, dart_fundamental_processor, fdr_info_processor)
fundamental_factor_df = fundamental_factor_processor.get_fundamental_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

factor_filter = FACTOR_FILTER(fundamental_factor_df)
fundamental_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FILTER"])

In [11]:
# Technical
## Moving Average
linear_coefficent_factor_processor = LINEAR_COEFFICENT_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
linear_coefficent_factors_df = linear_coefficent_factor_processor.get_linear_coef_factor_df(
    CFG["LINEAR_COEF"]["PROCESSOR"]
)

factor_filter = FACTOR_FILTER(linear_coefficent_factors_df)
technical_stockcodes = factor_filter.get_filtered_stockcodes(CFG["LINEAR_COEF"]["FILTER"], fundamental_stockcodes)

In [12]:
# Trader
pykrx_loader = get_pykrx_loader()

trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)
trader_factor_df = trader_factor_processor.get_trader_factor_df(technical_stockcodes, CFG["TRADER"]["PROCESSOR"])

factor_filter = FACTOR_FILTER(trader_factor_df)
final_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FILTER"], technical_stockcodes)

In [13]:
final_stockcodes

{'250000'}